In [2]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from typing import Optional, Any, List, Union, TypeVar, Type, cast, Callable


from pprint import pprint


def printText(block, path):
    array = block.select(path)
    print(len(array))
    del array[0]
    for p2para in array:
        print(" %%%" + p2para.text)


def p5(block):
    print(block.select('Title')[0].text)
    for p in block.select('P'):
        print("++" + p.find('Text').text)
        for unorderedlist in p.select('UnorderedList'):
            for listItem in unorderedlist.find_all('ListItem'):
                print(" " + listItem.find('Para').text)
                for orderList in listItem.find_all('OrderedList'):
                    for listItem1 in orderList.find_all('ListItem'):
                        if (listItem1.has_attr('NumberOverride')):
                            print(" " + listItem1['NumberOverride'])
                        print("  " + listItem1.find('Para').text)
        for p5 in p.select('P5'):
            print(p5.find('Pnumber').text)
            for p5para in p5.select('P5para'):
                print(p5para.find('Text').text)


url = 'http://www.legislation.gov.uk/ukpga/2004/34/section/76/data.xml'
response = requests.get(url)
soup = BeautifulSoup(response.text, "xml")
p5condition = False
lists = False

for part in soup.find_all('Part'):
    print(part.find('Number').text + " " + part.find('Title').text)
    # Chapter 4, 6
    if (len(part.select('Pblock > P1group')) == 0):
        path = 'P1group'
    else:
        path = 'Pblock > P1group'
    for first in part.select(path):
        print("  " + first.select('Pnumber')[0].text + ". " + first.find('Title').text)
        p1 = first.findAll("P1")
        if p1[0].has_attr('DocumentURI'):
            print(p1[0]['DocumentURI'])
            if first.select('P1 > P1para >Text'):
                print(first.select('P1 > P1para >Text')[0].text)
                print("")
            p1paras = first.select('P1para');
            blocks = first.select('BlockAmendment > P1group > P1 > P1para');
            topPara1s = len(p1paras) - len(blocks)
            for p1para in p1paras[:topPara1s]:
                if (p1para.find('P2') == None):
                    elements = []
                elif (len(p1para.select('P5'))):
                    p5condition = True
                    elements = p1para.find_all_next()
                else:
                    elements = p1para.find_all_next()
                for element in elements:
                    if (element.name == 'P2'):
                        if (element.has_attr('DocumentURI')):
                            print("  " + element.select('Pnumber')[0].text)
                            print("  " + element.select('P2para > Text')[0].text)
                            p2 = element;
                            if (len(p2.select('P2 > P2para > BlockAmendment > P1group')) != 0 and not p5condition):
                                for block in p2.select('P2 > P2para > BlockAmendment > P1group'):
                                    print("  " + block.select('P1 > Pnumber')[0].text)
                                    print("  " + block.select('Title')[0].text)
                                    textIndex = 0
                                    for p1 in block.select('P1 > P1para > P2'):
                                        print("   " + p1.select('Pnumber')[0].text + " " + p1.find('Text').text)
                                        # 185
                                        p3s = p1.select('P2 > P2para > P3')
                                        final_text = ""
                                        for p3 in p3s:
                                            print("  " + p3.select('Pnumber')[0].text + " " + p3.find('Text').text)
                                            final_text = p3.find_next_sibling('Text')
                                            for p4 in p3.select('P3 > P3para > P4'):
                                                print(
                                                    " " + p4.select('Pnumber')[0].text + " " + p4.find('Text').text)
                                                array = block.select('P2 > P2para > Text')
                                                del array[0]
                                                for p2para in array:
                                                    print("  " + p2para.text)
                                        if (type(final_text) is str):  # 185, 179
                                            print(final_text)
                                        elif (hasattr(final_text, 'text')):
                                            print(final_text.text)
                            elif (len(p2.select('P5')) != 0):
                                p5(p2)
                            elif (len(p2.select(
                                    'P2 > P2para > BlockAmendment')) != 0):  # 211( Text -> P3), #5 (P2 -> P2Para),#185
                                for block in p2.select('P2 > P2para > BlockAmendment'):
                                    textArray = block.select('Text')
                                    if (len(textArray) > 0):
                                        for p3 in block.select('BlockAmendment > P3'):
                                            print("  " + p3.select('Pnumber')[0].text + " " + p3.find(
                                                'Text').text)
                                        for p2 in block.select('BlockAmendment > P2'):
                                            print("   " + p2.select('Pnumber')[0].text + " " + p2.find(
                                                'Text').text)
                            elif (len(p2.select('P2 > P2para > P3')) != 0):
                                if (p2.has_attr('DocumentURI')):
                                    for p3 in p2.select('P2 > P2para > P3'):
                                        print("   " + p3.select('Pnumber')[0].text + " " + p3.find('Text').text)
                                        for p4 in p3.select('P3 > P3para > P4'):
                                            # PNUMBER and text display
                                            print(
                                                "  " + p4.select('Pnumber')[0].text + " " + p4.find('Text').text)

                                    textArray = p3.select('P3 > P3para >Text')  # 5
                                    del textArray[0]
                                    for p3para in textArray:
                                        print("  " + p3para.text)

                                    array = p2.select('P2 > P2para > Text')
                                    del array[0]
                                    for p2para in array:
                                        print(" " + p2para.text)

                            if (len(p2.find_all('UnorderedList')) > 0):
                                lists = True
                            for unorderedlist in p2.find_all('UnorderedList'):
                                for listItem in unorderedlist.select('UnorderedList > ListItem'):
                                    print("  " + listItem.find('Para').text)
                                    for orderList in listItem.select('UnorderedList > ListItem > OrderedList'):
                                        for listItem1 in orderList.find_all('ListItem'):
                                            if (listItem1.has_attr('NumberOverride')):
                                                print(" " + listItem1['NumberOverride'])
                                            print(" " + listItem1.find('Para').text)
                    elif (element.name == 'UnorderedList' and not p5condition and not lists):  # 98, dublication, 198
                        if (type(p1).__name__ == 'ResultSet'):
                            path = p1[0]
                        else:
                            path = p1
                        for unorderedlist in path.find_all('UnorderedList'):
                            for listItem in unorderedlist.find_all('ListItem'):
                                print("  " + listItem.find('Para').text)
                                for orderList in listItem.find_all('OrderedList'):
                                    for listItem1 in orderList.find_all('ListItem'):
                                        if (listItem1.has_attr('NumberOverride')):
                                            print(" " + listItem1['NumberOverride'])
                                        print("  " + listItem1.find('Para').text)
                    elif (element.name == 'BlockText'):
                        for blockText in p1[0].select('BlockText > Para > Text'):
                            print(" " + blockText.text)
        if (first.select('P1para > P2') == None):
            for P1para in first.select('P1para'):
                print(" " + P1para.find('Text').text)

        if (lists):
            for unorderedlist in first.select('P1para > UnorderedList'):
                for listItem in unorderedlist.find_all('ListItem'):
                    print("  " + listItem.find('Para').text)
                    for orderList in listItem.find_all('OrderedList'):
                        for listItem1 in orderList.find_all('ListItem'):
                            if (listItem1.has_attr('NumberOverride')):
                                print(" " + listItem1['NumberOverride'])
                            print("  " + listItem1.find('Para').text)

        if (len(first.select('P1 > P1para > P2')) == 0):
            x = len(first.find_all('Text')) - 1
            print(first.find_all('Text')[x].text)



Part 2 Licensing of houses in multiple occupation
  76. Transitional arrangements relating to introduction and termination of licensing
http://www.legislation.gov.uk/ukpga/2004/34/section/76
  1
  Subsection (2) applies where—
   a an order under section 55(3) which prescribes a particular description of HMOs comes into force; or
   b a designation under section 56 comes into force in relation to HMOs of a particular description.
  2
  This Part applies in relation to the occupation by persons or households of such HMOs on or after the coming into force of the order or designation even if their occupation began before, or in pursuance of a contract made before, it came into force.
  3
  Subsection (4) applies where—
   a an HMO which is licensed under this Part, or a part of such an HMO, is occupied by more households or persons than the number permitted by the licence; and
   b the occupation of all or any of those households or persons began before, or in pursuance of a contract made